In [ ]:
%matplotlib inline
import numpy as np
import skimage.io
import matplotlib.pyplot as plt
import skimage.morphology
import os
import utils.evaluation
import pandas as pd
import seaborn as sb

In [ ]:
base_dir = "/data1/image-segmentation/BBBC022/"

dir_gtruth = base_dir + "raw_annotations/"

tag = "01"

out_dir_label = base_dir + "unet/experiments/" + tag + "/out/segm/"

In [ ]:
def show(ground_truth, prediction, IOU):
    fig, ax = plt.subplots(1, 4, figsize=(18,6))
    ax[0].imshow(ground_truth)
    A = ground_truth.copy()
    B = prediction.copy()
    A[A > 0] = 1
    B[B > 0] = 1
    diff = A - B
    ax[1].imshow(diff)
    ax[2].imshow(prediction)
    ax[3].imshow(IOU)

In [ ]:
def compute_results(ground_truth, prediction, results):
    # Compute Intersection over Union
    IOU = utils.evaluation.intersection_over_union(ground_truth, prediction)
    show(ground_truth, prediction, IOU)

    # Calculate precision at all thresholds
    for t in np.arange(0.5, 1.0, 0.05):
        p = utils.evaluation.precision_at(t, IOU)
        res = {"Image": image_name, "Threshold": t, "Precision": p}
        row = len(results)
        results.loc[row] = res
        
    return results

In [ ]:
all_images = os.listdir(out_dir_label)

results = pd.DataFrame(columns=["Image", "Threshold", "Precision"])

for image_name in all_images:
    # Load ground truth data
    ground_truth = skimage.io.imread(dir_gtruth + image_name)
    # Transform to label matrix
    ground_truth = skimage.morphology.label(ground_truth[:,:,0])
    # Load predictions
    prediction = skimage.io.imread(out_dir_label + image_name)
    
    results = compute_results(ground_truth, prediction, results)



In [ ]:
average_precision = results.groupby("Threshold").mean().reset_index()
average_precision

In [ ]:
sb.regplot(data=average_precision, x="Threshold", y="Precision", order=3, ci=None)

In [ ]:
mean_average_precision = average_precision["Precision"].mean()
print("MAP:", mean_average_precision)